<a href="https://colab.research.google.com/github/sydneydeardorffnb/proserv/blob/main/API%20V2%20Tag%20Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation
Installing necessary libraries


In [ ]:
!pip install ratelim pandas tenacity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install google-api-python-client==1.12.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [ ]:
!pip install --upgrade google-auth==1.25.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade --quiet gspread


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import numpy as np

In [ ]:
import requests

In [ ]:
import pprint

# Define Variables
Enter your slug, doc name and API key

In [ ]:
worksheet = gc.open('Downloaded Tags').sheet1

#Sheet should start off completely blank.

In [ ]:
nationSlug = 'yourslughere'

In [ ]:
apiToken = 'yourtokenhere'
#You can get this via the developer tab in the nation settings. 
#If that tab is not available, you will need to enable Developer Tools Access in 3dna.
#For this script, the API token must be a non-expired v2 API token

# Do The Thing

In [ ]:
api_key = apiToken

In [ ]:
  matchurl = ('https://' + nationSlug + '.nationbuilder.com/api/v2/signup_tags?page[size]=100'+ '&access_token=' + api_key)
  recipientResponse = requests.get(matchurl)
  tagsResponse = recipientResponse.json()
  print(recipientResponse.json())

In [ ]:
import pandas as pd

import json
# load data using Python JSON module
df = pd.json_normalize(tagsResponse,record_path=['data'])
print(df)

In [ ]:
import json
nextUrl = tagsResponse['links']['next']
print(nextUrl)
relString = json.dumps(tagsResponse)
while "tags" in relString:
  nextUrl = tagsResponse['links']['next']
  fullNextUrl = ('https://' + nationSlug + '.nationbuilder.com' + nextUrl)
  print(fullNextUrl)
  print("Go to next page")
  matchurl = fullNextUrl
  recipientResponse = requests.get(matchurl)
  tagsResponse = recipientResponse.json()
  print(tagsResponse)
  relString = json.dumps(tagsResponse)
  print(relString)
  if "tags" in relString:
    df2 = pd.json_normalize(tagsResponse,record_path=['data'])
    df = df.append(df2)


In [ ]:

df3 = df #add new columns to be filled in by upcoming API requests
df3.insert(3,'tag_id',' ')
df3.insert(5,'tag_name',' ')
df3.insert(4,'tag_count',' ')
print(df3)

In [ ]:
df3 = df3.reset_index()
for index, row in df3.iterrows():
  tagGet = ('https://' + nationSlug + '.nationbuilder.com/api/v2/signup_tags/'+ row['id'] + '?fields[signup]=name,taggings_count&access_token=' + api_key)
  print(tagGet)
  tagResponse = requests.get(tagGet)
  tagJson = tagResponse.json()
  print(tagJson)
  tagId = tagJson['data']['id']
  tagName = tagJson['data']['attributes']['name']
  tagCount = tagJson['data']['attributes']['taggings_count']
  print(tagId)
  df3.at[index,'tag_name'] = tagName
  df3.at[index,'tag_count'] = tagCount
  df3.at[index,'tag_id'] = tagId
  #df3 = df3.drop(df3.columns[[0,1,2]], axis=1) #Delete useless columns that come from redundant json garbage
  print(df3)
  

In [ ]:
worksheet.update([df3.columns.values.tolist()] + df3.values.tolist())
